# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' + nn + '_layer_outputs.dat', 'rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()


# Load recording and test data

In [2]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 1
if factor<1:
    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255


Using Theano backend.


# Run Neural Networks and average outputs

In [3]:
from copy import copy, deepcopy
from RE_PartialRecData2 import RE_PartialRecData2
from sklearn import preprocessing, linear_model
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping
#from keras import backend as K
#import tensorflow as tf
import time

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
nSubnetSize = len(subnetSize)
# which layers?
#iLayer = 0
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for iLayer in range(2, len(layer_outs)-1,2):
    print('Layer #', iLayer)
    for ss in range(nSubnetSize):
        #r_coefs = np.zeros((nIterations, 10, 128))
        #nLayerNeurons = [subnetSize[ss], 0, 0, 0, 10]
        nLayerNeurons = subnetSize[ss]
        print(subnetSize[ss])    
        for it in range(nIterations):
            start = time.time()
            # copy data - is this necessary?
            layer_outputs = deepcopy(layer_outs)
            rmses_rec = np.zeros((nRecordings, nOutNeurons))
            for iRec in range(nRecordings):
                # subsample
                X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                              nLayerNeurons, 1, nSamples[ss])
                # impute X_subsample with mean value, apply imputation to test set
                imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
                impf = imp.fit(X_subsample)
                X_new = impf.transform(X_subsample)
                X_test_new = impf.transform(layer_outs_test[iLayer])
                # fit ΝΝ
                model = Sequential()
                model.add(Dense(16, input_shape=(X_new.shape[1],), activation='relu'))
                model.add(Dense(16, activation='relu'))
                model.add(Dense(16, activation='relu'))                
                model.add(Dense(10, activation='softmax'))
                # Compile model
                model.compile(loss='mean_squared_error', optimizer='adam')
                callbacks = [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0)]
                model.fit(X_new, Y_subsample, nb_epoch=30, batch_size=16, shuffle=True, verbose=0, 
                          validation_split=0.1, callbacks=callbacks)

                rmses_rec[iRec, :] = np.sqrt(np.mean((model.predict(X_test_new) - layer_outs_test[oLayer])**2, axis=0))
            stop = time.time()
            duration = stop-start
            print(duration)
            rmses[it,:,ss] = np.mean(rmses_rec, axis=0)

            print ('predicting, mean RMSEs=%f' %np.mean(rmses[it, :, ss]))
    fName = 'results/NNMP_RMSES_Layer' + str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()
    

('Layer #', 2)
1


/opt/anaconda/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


236.874207973
predicting, mean RMSEs=0.282033
257.694216967
predicting, mean RMSEs=0.288554
236.35535717
predicting, mean RMSEs=0.287188
257.990689993
predicting, mean RMSEs=0.285514
269.890087843
predicting, mean RMSEs=0.285521
2
179.876955032
predicting, mean RMSEs=0.267442
181.177634001
predicting, mean RMSEs=0.266678
151.465945005
predicting, mean RMSEs=0.265589
166.893733025
predicting, mean RMSEs=0.271509
162.076573133
predicting, mean RMSEs=0.272831
4
116.439636946
predicting, mean RMSEs=0.260336
102.191302061
predicting, mean RMSEs=0.241246
119.271636009
predicting, mean RMSEs=0.240004
113.86674118
predicting, mean RMSEs=0.248563
113.195714951
predicting, mean RMSEs=0.248644
8
84.4532051086
predicting, mean RMSEs=0.194669
85.8731939793
predicting, mean RMSEs=0.187588
81.5252099037
predicting, mean RMSEs=0.208254
78.9892151356
predicting, mean RMSEs=0.177570
73.0041861534
predicting, mean RMSEs=0.190318
16
49.3108201027
predicting, mean RMSEs=0.121354
53.3264780045
predicting, m